In [ ]:
import numpy as np
import pandas as pd

In [ ]:
crime_anal_police = pd.read_csv('../data/02.crime_in_Seoul.csv', thousands=',', encoding='euc-kr')

In [ ]:
crime_anal_police.head()

In [ ]:
del crime_anal_police["Unnamed: 11"]

In [ ]:
crime_anal_police.head()

In [ ]:
# import googlemaps
# gmaps_key = "*map key*"
# gmaps = googlemaps.Client(key=gmaps_key)
# gmaps.geocode('서울중부경찰서', language='ko')

In [ ]:
# station_name = ['서울'+str(name[:-1])+'경찰서' for name in crime_anal_police['관서명']]

In [ ]:
# station_name

In [ ]:
# station_geocode = [gmaps.geocode(name, language='ko') for name in station_name]

In [ ]:
# station_address = [geocode[0]["formatted_address"] for geocode in station_geocode]

In [ ]:
# station_lat = [geocode[0]["geometry"]["location"]['lat'] for geocode in station_geocode]

In [ ]:
# station_lng = [geocode[0]["geometry"]["location"]['lng'] for geocode in station_geocode]

In [ ]:
# df_station_geo_data = pd.DataFrame(data={'station_name': station_name, 'station_address': station_address,
#                                          'station_lat': station_lat, 'station_lng': station_lng} )

# df_station_geo_data.to_csv('../data/02.station_geo_data_by_googlemaps.csv', sep=',', encoding='utf-8')

In [ ]:
df_station_geo_data_by_googlemaps = pd.read_csv('../data/02.station_geo_data_by_googlemaps.csv',
                                                encoding='utf-8', index_col=0)

In [ ]:
df_station_geo_data_by_googlemaps

In [ ]:
# gu_name = []
# for name in station_address:
#     tmp = name.split()
#     tmp_gu = [gu for gu in tmp if gu[-1] == '구'][0]
#     gu_name.append(tmp_gu)

In [ ]:
# crime_anal_police['구별'] = gu_name

In [ ]:
# crime_anal_police.head()

In [ ]:
# crime_anal_police[crime_anal_police['관서명']=='금천서']

In [ ]:
# crime_anal_police.loc[crime_anal_police['관서명']=='금천서', ['구별']]

In [ ]:
# crime_anal_police.loc[crime_anal_police['관서명']=='금천서', ['구별']] = '금천구'

In [ ]:
# crime_anal_police[crime_anal_police['관서명']=='금천서']

In [ ]:
# crime_anal_police.to_csv('../data/02.crime_in_Seoul_include_gu_name_20180315.csv', sep=',', encoding='utf-8')

In [ ]:
crime_anal_raw = pd.read_csv('../data/02.crime_in_Seoul_include_gu_name.csv',
                             encoding='utf-8', index_col=0)

In [ ]:
crime_anal_raw

In [ ]:
crime_anal = pd.pivot_table(crime_anal_raw, index='구별', aggfunc=np.sum)

In [ ]:
crime_anal

In [ ]:
crime_anal['강간검거율'] = crime_anal['강간 검거']/crime_anal['강간 발생']*100
crime_anal['강도검거율'] = crime_anal['강도 검거']/crime_anal['강도 발생']*100
crime_anal['살인검거율'] = crime_anal['살인 검거']/crime_anal['살인 발생']*100
crime_anal['절도검거율'] = crime_anal['절도 검거']/crime_anal['절도 발생']*100
crime_anal['폭력검거율'] = crime_anal['폭력 검거']/crime_anal['폭력 발생']*100

In [ ]:
crime_anal

In [ ]:
del crime_anal['강간 검거']
del crime_anal['강도 검거']
del crime_anal['살인 검거']
del crime_anal['절도 검거']
del crime_anal['폭력 검거']

In [ ]:
crime_anal

In [ ]:
con_list = ['강간검거율', '강도검거율', '살인검거율', '절도검거율', '폭력검거율']
for con in con_list:
    crime_anal.loc[crime_anal[con] > 100, con] = 100

In [ ]:
crime_anal

In [ ]:
crime_anal.rename(columns = {'강간 발생': '강간',
                             '강도 발생': '강도',
                             '살인 발생': '살인',
                             '절도 발생': '절도',
                             '폭력 발생': '폭력'}, inplace=True)

In [ ]:
crime_anal

In [ ]:
from sklearn import preprocessing
col = ['강간','강도', '살인', '절도', '폭력']
x = crime_anal[col].values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x.astype(float))
crime_anal_norm = pd.DataFrame(x_scaled, columns=col, index=crime_anal.index)
col2 = ['강간검거율', '강도검거율', '살인검거율', '절도검거율', '폭력검거율']
crime_anal_norm[col2] = crime_anal[col2]

In [ ]:
crime_anal_norm

In [ ]:
result_CCTV = pd.read_csv('../data/01.CCTV_result.csv', encoding='UTF-8', index_col='구별')

In [ ]:
result_CCTV

In [ ]:
crime_anal_norm[['인구수', 'CCTV']] = result_CCTV[['인구수', '소계']]

In [ ]:
crime_anal_norm

In [ ]:
col = ['강간','강도', '살인', '절도', '폭력']
crime_anal_norm['범죄'] = np.sum(crime_anal_norm[col], axis=1)

In [ ]:
crime_anal_norm

In [ ]:
col = ['강간검거율', '강도검거율', '살인검거율', '절도검거율', '폭력검거율']
crime_anal_norm['검거'] = np.sum(crime_anal_norm[col], axis=1)

In [ ]:
crime_anal_norm

In [ ]:
import platform
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['axes.unicode_minus'] = False
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    path = 'C:Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system')
%matplotlib inline

In [ ]:
sns.pairplot(crime_anal_norm, vars=['강도', '살인', '폭력'],
             kind='reg', size=3)
plt.show()

In [ ]:
 sns.pairplot(crime_anal_norm, x_vars=['인구수', 'CCTV'],
              y_vars=['살인', '강도'], kind='reg', size=3)
plt.show()

In [ ]:
sns.pairplot(crime_anal_norm,
             x_vars=['인구수', 'CCTV'],
             y_vars=['살인검거율', '폭력검거율'],
             kind='reg', size=3)
plt.show()

In [ ]:
tmp_max = crime_anal_norm['검거'].max()
crime_anal_norm['검거'] = crime_anal_norm['검거'] / tmp_max * 100
crime_anal_norm_sort = crime_anal_norm.sort_values(by='검거', ascending=False)
crime_anal_norm_sort

In [ ]:
target_col = ['강간검거율', '강도검거율', '살인검거율', '절도검거율', '폭력검거율']
plt.figure(figsize = (10, 10))
sns.heatmap(crime_anal_norm_sort[target_col], annot=True, fmt='f', linewidths=.5)
plt.title('범죄 검거 비율(정규화된 검거의 합으로 정렬)')
plt.show()

In [ ]:
target_col = ['강간', '강도', '살인', '절도', '폭력', '범죄']
crime_anal_norm['범죄'] = crime_anal_norm['범죄'] / 5
crime_anal_norm_sort = crime_anal_norm.sort_values(by='범죄', ascending=False)
plt.figure(figsize=(10, 10))
sns.heatmap(crime_anal_norm_sort[target_col], annot=True, fmt='f', linewidths=.5)
plt.title('범죄비율(정규화된 발생 건수로 정렬)')
plt.show()

In [ ]:
crime_anal_norm.to_csv('../data/02.crime_in_Seoul_final_20180315.csv',
                       sep=',', encoding='utf=8')

In [ ]:
import json
geo_path = '../data/02.skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))
geo_str

In [ ]:
import folium
map = folium.Map(location=[37.5502, 126.982], zoom_start=11,
                 tiles='Stamen Toner')
map.choropleth(geo_data = geo_str,
               data = crime_anal_norm['살인'],
               columns = [crime_anal_norm.index, crime_anal_norm['살인']],
               fill_color = 'PuRd',
               key_on = 'feature.id')
map

In [ ]:
import folium
map = folium.Map(location=[37.5502, 126.982], zoom_start=11,
                 tiles='Stamen Toner')
map.choropleth(geo_data = geo_str,
               data = crime_anal_norm['강간'],
               columns = [crime_anal_norm.index, crime_anal_norm['강간']],
               fill_color = 'PuRd',
               key_on = 'feature.id')
map

In [ ]:
import folium
map = folium.Map(location=[37.5502, 126.982], zoom_start=11,
                 tiles='Stamen Toner')
map.choropleth(geo_data = geo_str,
               data = crime_anal_norm['범죄'],
               columns = [crime_anal_norm.index, crime_anal_norm['범죄']],
               fill_color = 'PuRd',
               key_on = 'feature.id')
map

In [ ]:
tmp_criminal = crime_anal_norm['살인'] / crime_anal_norm['인구수'] * 1000000
map = folium.Map(location=[37.5502, 126.982], zoom_start=11,
                 tiles='Stamen Toner')
map.choropleth(geo_data = geo_str,
               data = tmp_criminal,
               columns = [crime_anal.index, tmp_criminal],
               fill_color = 'PuRd',
               key_on = 'feature.id')
map

In [ ]:
crime_anal_raw['lat'] = df_station_geo_data_by_googlemaps['station_lat']
crime_anal_raw['lng'] = df_station_geo_data_by_googlemaps['station_lng']

In [ ]:
crime_anal_raw

In [ ]:
col = ['살인 검거', '강도 검거', '강간 검거', '절도 검거', '폭력 검거']
tmp = crime_anal_raw[col] / crime_anal_raw[col].max()
crime_anal_raw['검거'] = np.sum(tmp, axis=1)
crime_anal_raw.head()

In [ ]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11)

for n in crime_anal_raw.index:
    folium.Marker([crime_anal_raw['lat'][n], crime_anal_raw['lng'][n]]).add_to(map)
map

In [ ]:
map1 = folium.Map(location=[37.5502, 126.982], zoom_start=11)

for n in crime_anal_raw.index:
    folium.CircleMarker([crime_anal_raw['lat'][n], crime_anal_raw['lng'][n]],
                        radius=int(crime_anal_raw['검거'][n]*10),
                        color='#3186cc', fill_color='#3186cc', fill=True).add_to(map1)
map1

In [ ]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11)

map.choropleth(geo_data = geo_str,
               data = tmp_criminal,
               columns = [crime_anal.index, tmp_criminal],
               fill_color = 'PuRd',
               key_on = 'feature.id')

for n in crime_anal_raw.index:
    folium.CircleMarker([crime_anal_raw['lat'][n], crime_anal_raw['lng'][n]],
                        radius=int(crime_anal_raw['검거'][n]*10),
                        color='#3186cc', fill_color='#3186cc', fill=True).add_to(map)

map